In [1]:
import pandas as pd
import numpy as np
from skbio.stats.composition import clr
import os
import sys
os.chdir('/home/alexis/UAM/picrust/DataSets/')

In [45]:
#Create Metabolic.net files for analysis in BiomeNet
#Metabolic net files have the format reactant,reaxionID,product

#Load reactions in the Metacyc Database, annotated with reactants and products
EC_reactions = pd.read_csv('/home/alexis/UAM/picrust/default_files/metacyc_rxns_v24.csv', sep = '\t', index_col = 'Reaction') 
EC_reactions.head()

,Right,Left,EC-Number
Reaction,,,
DEOXYNUCLEOTIDE-3-PHOSPHATASE-RXN,Deoxy-Ribonucleosides Pi,Deoxy-Ribonucleoside-3P WATER,EC:3.1.3.34
RXN-20295,CPD-21889 CARBON-DIOXIDE WATER Oxidized-ferred...,CPD-21911 OXYGEN-MOLECULE Reduced-ferredoxins ...,EC:1.14.15
RXN-11130,CPD-12101 CO-A,CPD-11571 ACETYL-COA,EC:2.3.1
HYPONITRITE-REDUCTASE-RXN,PROTON CPD-1541 NADH,HYDROXYLAMINE NAD,EC:1.7.1.5
RXN-19187,CPD-20683 ADP PROTON,CPD-20682 ATP,EC:2.7.1.M16


In [96]:
metagenome_unstrat = pd.read_csv('EC_metagenome_out/pred_metagenome_unstrat.tsv.gz', compression = 'gzip', sep = '\t')
metagenome_unstrat.head()

,function,SRR2071024,SRR2071075,SRR2029458,SRR2029459,SRR2029460,SRR7135742,SRR7135743,CIR_18,IL_18,...,PR9_18,ALC_19,CIR_19,ERR844085,ERR844086,ALC_20,My_20,Pty_20,V3_20,VT_20
0,EC:1.1.1.1,569327.80,1358307.09,32853.85,31431.34,32776.49,108609.34,114356.08,2119.0,8145.25,...,7749.0,16234.75,2726.0,79297.66,232808.34,54481.17,45108.65,38240.35,24364.68,92869.4
1,EC:1.1.1.100,1595044.88,4017067.99,75906.92,85869.36,91134.96,316157.74,319596.12,22631.5,27922.25,...,16719.0,30634.50,71197.0,178744.34,117803.02,106479.34,77172.98,144959.02,81021.79,138866.6
2,EC:1.1.1.102,0.00,0.00,0.00,41.00,7.50,0.00,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3,EC:1.1.1.103,76164.33,222163.72,423.42,377.50,1257.00,20822.83,16663.33,176.0,1236.50,...,2846.0,230.75,181.0,2761.33,33.67,11111.00,2484.00,5552.00,7429.00,0.0
4,EC:1.1.1.105,120.00,790.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [97]:
#Create the files for metabolic divergence analysis between samples
#For each sample in the metagenome_file
missing_ec = []
for sample in metagenome_unstrat.iloc[:,1:].columns:
    #Create a metabolic.net file
    metabolic_net = open('/home/alexis/BiomeNet/BiomeNet/metabolic_nets/DataSets/'+sample+'.metabolic.net','w')
    #Filter the reactions with non-zero abundance
    nonzero_filter = metagenome_unstrat.loc[:, sample] > 0
    temp_df = metagenome_unstrat.loc[nonzero_filter,['function', sample]]
    temp_df.loc[:, sample] = clr(temp_df.loc[:, sample])
    temp_df.loc[:, sample] = temp_df.loc[:, sample] - 5*temp_df.loc[:, sample].min()
    #For each non-zero abundance reaction
    n = 0
    for index in temp_df.index:
        #Filter all reactions in the metacyc database annotated with indexed EC-number in metagenome
        EC_filter = EC_reactions.loc[:, 'EC-Number'] == temp_df.loc[index, 'function']
        if not EC_reactions.loc[EC_filter].index.empty:
            #For all reactions wich matched the EC-number
            for rxn in EC_reactions.loc[EC_filter].index:
                #repeat the process n times, where n is the abundace of the indexed reaction
                for i in range(int(round(temp_df.loc[index, sample]))):
                    #Map reactant,reaxionID,product
                    for reactant in EC_reactions.loc[rxn, 'Right'].split():
                        for product in EC_reactions.loc[rxn, 'Left'].split():
                            metabolic_net.write(reactant+','+rxn+','+product+'\n')
                    metabolic_net.write('\n')
        else:
            if temp_df.loc[index, 'function'] in missing_ec:
                pass
            else:
                n += 1
                missing_ec.append(temp_df.loc[index, 'function'])
    print(n)
    del temp_df
    metabolic_net.close()

17
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [98]:
#Load stratified metagenome (metagenome by taxon contribution)
strat_path = '/home/alexis/UAM/qiime/Mar20/picrust2/EC_metagenome_out/pred_metagenome_contrib.tsv.gz'
metagenome_strat = pd.read_csv(strat_path, compression = 'gzip', sep = '\t')
metagenome_strat.head()

,sample,function,taxon,taxon_abun,taxon_rel_abun,genome_function_count,taxon_function_abun,taxon_rel_function_abun
0,ALC_19,EC:1.1.1.1,0746d4ed931f7a6c799e7b1e5f525cbc,29.0,0.642873,1,29.0,0.642873
1,ALC_19,EC:1.1.1.1,108039c87f18b4ab5b726b4417cefdca,473.0,10.485480,2,946.0,20.970960
2,ALC_19,EC:1.1.1.1,16d4ee39d4468351b3a62dbd64d29ed9,38.0,0.842385,1,38.0,0.842385
3,ALC_19,EC:1.1.1.1,2a8b9bb6fab9f06da548a13f0742c437,5.0,0.110840,3,15.0,0.332521
4,ALC_19,EC:1.1.1.1,419f4f6586d5b066c1c6361376f1c689,8.5,0.188428,1,8.5,0.188428


In [99]:
#Create a ditcionary of dataframes for each sample
sample = {}
#for each sample in the pre_metagenome
for smpl in metagenome_strat.loc[:,'sample'].unique():
    #Filter the taxons corresponding to sample i
    sample_filter = metagenome_strat.loc[:, 'sample'] == smpl
    #Create a dataframe of size = functions x taxons
    taxons = metagenome_strat.loc[sample_filter, 'taxon'].unique()
    functions = metagenome_strat.loc[sample_filter, 'function'].unique()
    sample[smpl] = pd.DataFrame(index = functions, columns  = taxons)
    del taxons
    del functions
    #for every entry in sample i
    for idx in metagenome_strat.loc[sample_filter].index:
        #query the function abundace
        function = metagenome_strat.loc[idx, 'function']
        #For every taxon
        taxon = metagenome_strat.loc[idx, 'taxon']
        #Write it to the corresponding position in the dataframe
        sample[smpl].loc[function, taxon] = metagenome_strat.loc[idx, 'taxon_function_abun'].copy()
        del function
        del taxon
    sample[smpl].index.name = 'function'
    sample[smpl].reset_index(inplace = True)
    sample[smpl].fillna(0, inplace = True)  
    #Create directory to save metabolic.net files
    #os.mkdir("/home/alexis/BiomeNet/BiomeNet/taxon_analysis/metabolic_nets/"+smpl)


In [101]:
missing_ec = []
for smpl in sample.keys():
    n = 0
    for taxon in sample[smpl].columns[1:]:
        #Create a metabolic.net file
        metabolic_net = open('/home/alexis/BiomeNet/BiomeNet/taxon_analysis/metabolic_nets/'+smpl+'/'+taxon+'.metabolic.net','w+')
        #Filter the reactions with non-zero abundance
        nonzero_filter = sample[smpl].loc[:, taxon] > 0
        temp_df = sample[smpl].loc[nonzero_filter,['function', taxon]].copy()
        #Calculate the centered log-ratio of non-zero reactions
        temp_df.loc[:, taxon] = clr(temp_df.loc[:, taxon])
        #translate the clr so that all values are positive (this is to be used as the abundance of reaction in metabolic.net files)
        temp_df.loc[:, taxon] = temp_df.loc[:, taxon] - 5*temp_df.loc[:, taxon].min()
        #For each non-zero abundance reaction
        for index in temp_df.index:
            #Filter all reactions in the metacyc database annotated with the indexed EC-number in metagenome
            EC_filter = EC_reactions.loc[:, 'EC-Number'] == temp_df.loc[index, 'function']
            if not EC_reactions.loc[EC_filter].index.empty:
                #For all reactions wich matched the EC-number
                for rxn in EC_reactions.loc[EC_filter].index:
                    #repeat the process n times, where n is the abundace of the indexed reaction
                    for i in range(int(round(temp_df.loc[index, taxon]))):
                        #Map reactant,reaxionID,product
                        for reactant in EC_reactions.loc[rxn, 'Right'].split():
                            for product in EC_reactions.loc[rxn, 'Left'].split():
                                metabolic_net.write(reactant+','+rxn+','+product+'\n')
                        metabolic_net.write('\n')
            else:
                if temp_df.loc[index, 'function'] in missing_ec:
                    pass
                else:
                    n += 1
                    missing_ec.append(temp_df.loc[index, 'function'])
        del temp_df
        metabolic_net.close()
    print(n)


12
0
1
0
0
0
1


In [103]:
import glob
path = r'/home/alexis/BiomeNet/BiomeNet/taxon_analysis/metabolic_nets/CIR_19/' # use your path
all_files = glob.glob(os.path.join(path,"*.metabolic.net"))
df = pd.concat((pd.read_csv(f, header = None) for f in all_files))
sample[smpl].loc[nonzero_filter,['function', taxon]]

2496